# PostgreSQL

In [57]:
import psycopg2

## Construct the Postgres function for checking connection, creating tables, deleting tables, and creating or dropping indexes

In [101]:
def pgsql(pgsql_cmd_str, kind):    
    try:
        # Connect to PostgreSQL
        connection = psycopg2.connect(user = "seanhong",
                                      password = "s017003",
                                      host = "localhost",
                                      database = "user_db")
        cursor = connection.cursor()
        if kind == "create":
            cursor.execute(pgsql_cmd_str)
            connection.commit()
            print("PostgreSQL table successfully created\n")
        elif kind == "drop":
            cursor.execute(pgsql_cmd_str)
            connection.commit()
            print("PostgreSQL table successfully dropped\n")
        elif kind == "connect":
            cursor.execute(pgsql_cmd_str)
            print("You are connected to - ", cursor.fetchone(),"\n")
        elif kind == "index":
            cursor.execute(pgsql_cmd_str)
            connection.commit()
            print("Indexes created/dropped successfully in PostgreSQL\n")
        else:
            print("Type the wrong kind of command. Only four commands are available: create, drop, connect, index")

    except (Exception, psycopg2.DatabaseError) as error :
        print ("Error while executing PostgreSQL commands:", error)
    finally:
        # Close PostgreSQL connection
            if(connection):
                cursor.close()
                connection.close()
                print("PostgreSQL connection is closed")

## Test the connection to PostgreSQL database

In [102]:
pgsql("SELECT version();", "connect")

You are connected to -  ('PostgreSQL 12.2 on x86_64-apple-darwin19.4.0, compiled by Apple clang version 11.0.3 (clang-1103.0.32.59), 64-bit',) 

PostgreSQL connection is closed


## Create the table

In [103]:
# Create table if not exists
pgsql(''' CREATE TABLE IF NOT EXISTS users
              (id BIGINT PRIMARY KEY,
              name TEXT NOT NULL,
              screen_name TEXT NOT NULL,
              protected BOOLEAN NOT NULL,
              verified BOOLEAN NOT NULL,
              followers_count BIGINT NOT NULL,
              friends_count INTEGER NOT NULL,
              listed_count INTEGER NOT NULL,
              favourites_count BIGINT NOT NULL,
              statuses_count BIGINT NOT NULL,
              created_at TEXT NOT NULL
              ); ''', "create")

PostgreSQL table successfully created

PostgreSQL connection is closed


## Insert or update data

In [104]:
# Connect to postgresql
connection = psycopg2.connect(user = "seanhong",
                              password = "s017003",
                              host = "localhost",
                              database = "user_db")
cursor = connection.cursor()
# Define a insert/update object
postgres_insert_one = '''
INSERT INTO 
    users
VALUES
    (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT (id)
    DO UPDATE SET
        (name, screen_name, protected, verified, followers_count, friends_count, listed_count, favourites_count, 
        statuses_count, created_at)
        = (EXCLUDED.name, EXCLUDED.screen_name, EXCLUDED.protected, EXCLUDED.verified, EXCLUDED.followers_count, 
        EXCLUDED.friends_count, EXCLUDED.listed_count, EXCLUDED.favourites_count, EXCLUDED.statuses_count, 
        EXCLUDED.created_at);'''    
# Load JSON file into postgresql database
import json
with open("twitter_hang", "r") as f1:
    for line in f1:
        try:
            data = json.loads(line)
            usr_dic = {}
            for key in data['user']:
                if key in ['id','name','screen_name','protected','verified','followers_count','friends_count',
                           'listed_count','favourites_count','statuses_count','created_at']:
                    usr_dic[key] = data['user'][key]            
            # INSERT data into postgre here
            user_to_insert = tuple(usr_dic.values())
            cursor.execute(postgres_insert_one, user_to_insert)
        except:
            continue
connection.commit()
# Close database connection.
if(connection):
    cursor.close()
    connection.close()
    print("Data inserted or updated successfully in user table\n")
    print("PostgreSQL connection is closed")

Data inserted or updated successfully in user table

PostgreSQL connection is closed


## Construct the querying function and analytical querying function

In [105]:
def query_data(pgsql_cmd_str, kind):
    try:
        # Connect to postgresql
        connection = psycopg2.connect(user = "seanhong",
                                      password = "s017003",
                                      host = "localhost",
                                      database = "user_db")
        cursor = connection.cursor()
        if kind == "result":            
            cursor.execute(pgsql_cmd_str)
            record = cursor.fetchall()
            if record == []:
                print("None of the users met the requirement")
            else:
                print("There are %d querying results"%(len(record)), "\n")
            for row in range(len(record)):
                print("NO.%d user"%(row+1))
                print("id :", record[row][0])
                print("name :", record[row][1])
                print("screen_name :", record[row][2])
                print("protected :", record[row][3])
                print("verified :", record[row][4])
                print("followers_count :", record[row][5])
                print("friends_count :", record[row][6])
                print("listed_count :", record[row][7])
                print("favourites_count :", record[row][8])
                print("statuses_count :", record[row][9])
                print("created_at :", record[row][10], "\n")
        elif kind == "analyze":
            cursor.execute(pgsql_cmd_str)
            record = cursor.fetchall()
            for obj in range(len(record)):
                print(record[obj],"\n")
        else:
            print("Type the wrong kind of command. Only two commands are available: result and analyze")
    except (Exception, psycopg2.DatabaseError) as error :
        print ("Error while querying data", error)
    # Close PostgreSQL connection
    finally:
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

## Analyze querying times before indexing

### 1. Find the top 3 most followed users as an example

In [106]:
query_data("EXPLAIN ANALYZE SELECT * FROM users ORDER BY followers_count DESC LIMIT 3", "analyze")

('Limit  (cost=673.09..673.10 rows=3 width=138) (actual time=5.835..5.836 rows=3 loops=1)',) 

('  ->  Sort  (cost=673.09..711.92 rows=15533 width=138) (actual time=5.834..5.835 rows=3 loops=1)',) 

('        Sort Key: followers_count DESC',) 

('        Sort Method: top-N heapsort  Memory: 25kB',) 

('        ->  Seq Scan on users  (cost=0.00..472.33 rows=15533 width=138) (actual time=0.033..4.136 rows=16045 loops=1)',) 

('Planning Time: 0.343 ms',) 

('Execution Time: 5.868 ms',) 

PostgreSQL connection is closed


### 2. Find users with numbers of followers more than 100K

In [107]:
query_data("EXPLAIN ANALYZE SELECT * FROM users WHERE followers_count>100000 ORDER BY followers_count DESC", "analyze")

('Sort  (cost=522.98..523.36 rows=150 width=100) (actual time=2.936..2.947 rows=134 loops=1)',) 

('  Sort Key: followers_count DESC',) 

('  Sort Method: quicksort  Memory: 44kB',) 

('  ->  Seq Scan on users  (cost=0.00..517.56 rows=150 width=100) (actual time=0.011..2.859 rows=134 loops=1)',) 

('        Filter: (followers_count > 100000)',) 

('        Rows Removed by Filter: 15911',) 

('Planning Time: 0.555 ms',) 

('Execution Time: 3.008 ms',) 

PostgreSQL connection is closed


### 3. Find the verified user with the smallest number of followers

In [108]:
query_data("EXPLAIN ANALYZE SELECT * FROM users WHERE verified = true ORDER BY followers_count ASC LIMIT 1", "analyze")

('Limit  (cost=478.76..478.76 rows=1 width=100) (actual time=2.600..2.600 rows=1 loops=1)',) 

('  ->  Sort  (cost=478.76..479.41 rows=261 width=100) (actual time=2.599..2.599 rows=1 loops=1)',) 

('        Sort Key: followers_count',) 

('        Sort Method: top-N heapsort  Memory: 25kB',) 

('        ->  Seq Scan on users  (cost=0.00..477.45 rows=261 width=100) (actual time=0.010..2.529 rows=261 loops=1)',) 

('              Filter: verified',) 

('              Rows Removed by Filter: 15784',) 

('Planning Time: 0.502 ms',) 

('Execution Time: 2.638 ms',) 

PostgreSQL connection is closed


## Create indexes for followers_count

In [109]:
pgsql("CREATE INDEX followers_idx ON users(followers_count)", "index")

Indexes created/dropped successfully in PostgreSQL

PostgreSQL connection is closed


## Create multicolumn indexes for followers and verified

In [110]:
pgsql("CREATE INDEX followers_verified_idx ON users(followers_count, verified)", "index")

Indexes created/dropped successfully in PostgreSQL

PostgreSQL connection is closed


## Analyze querying times after indexing

### 1. Find the top 3 most followed users as an example

In [111]:
query_data("EXPLAIN ANALYZE SELECT * FROM users ORDER BY followers_count DESC LIMIT 3", "analyze")

('Limit  (cost=0.29..0.60 rows=3 width=100) (actual time=0.022..0.030 rows=3 loops=1)',) 

('  ->  Index Scan Backward using followers_idx on users  (cost=0.29..1692.81 rows=16045 width=100) (actual time=0.021..0.029 rows=3 loops=1)',) 

('Planning Time: 0.593 ms',) 

('Execution Time: 0.060 ms',) 

PostgreSQL connection is closed


### 2. Find users with numbers of followers more than 100K

In [112]:
query_data("EXPLAIN ANALYZE SELECT * FROM users WHERE followers_count>100000 ORDER BY followers_count DESC", "analyze")

('Sort  (cost=273.69..274.06 rows=150 width=100) (actual time=0.406..0.414 rows=134 loops=1)',) 

('  Sort Key: followers_count DESC',) 

('  Sort Method: quicksort  Memory: 44kB',) 

('  ->  Bitmap Heap Scan on users  (cost=5.45..268.27 rows=150 width=100) (actual time=0.041..0.346 rows=134 loops=1)',) 

('        Recheck Cond: (followers_count > 100000)',) 

('        Heap Blocks: exact=107',) 

('        ->  Bitmap Index Scan on followers_verified_idx  (cost=0.00..5.41 rows=150 width=0) (actual time=0.027..0.027 rows=134 loops=1)',) 

('              Index Cond: (followers_count > 100000)',) 

('Planning Time: 0.633 ms',) 

('Execution Time: 0.477 ms',) 

PostgreSQL connection is closed


### 3. Find the verified user with the smallest number of followers

In [113]:
query_data("EXPLAIN ANALYZE SELECT * FROM users WHERE verified = true ORDER BY followers_count ASC LIMIT 1", "analyze")

('Limit  (cost=0.29..4.57 rows=1 width=100) (actual time=0.325..0.325 rows=1 loops=1)',) 

('  ->  Index Scan using followers_verified_idx on users  (cost=0.29..1119.17 rows=261 width=100) (actual time=0.324..0.324 rows=1 loops=1)',) 

('        Index Cond: (verified = true)',) 

('Planning Time: 0.539 ms',) 

('Execution Time: 0.355 ms',) 

PostgreSQL connection is closed


## Query the data

### Count the total number of users in the table

In [114]:
try:
    connection = psycopg2.connect(user = "seanhong",
                                  password = "s017003",
                                  host = "localhost",
                                  database = "user_db")
    cursor = connection.cursor("SELECT COUNT(*) FROM users;")
    pgsql_str = "SELECT COUNT(*) FROM users;"
    cursor.execute(pgsql_str)
    result = cursor.fetchone()
    print("Total number of users in the table:",result[0],"\n")
except (Exception, psycopg2.DatabaseError) as error :
    print ("Error while querying data", error)
finally:
    if(connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

Total number of users in the table: 16045 

PostgreSQL connection is closed


### 1. Find the top 3 most followed users as an example

In [115]:
query_data("SELECT * FROM users ORDER BY followers_count DESC LIMIT 3", "result")

There are 3 querying results 

NO.1 user
id : 1115874631
name : CGTN
screen_name : CGTNOfficial
protected : False
verified : True
followers_count : 14024195
friends_count : 56
listed_count : 8412
favourites_count : 68
statuses_count : 117674
created_at : Thu Jan 24 03:18:59 +0000 2013 

NO.2 user
id : 37034483
name : NDTV
screen_name : ndtv
protected : False
verified : True
followers_count : 12616711
friends_count : 15
listed_count : 12732
favourites_count : 0
statuses_count : 704225
created_at : Fri May 01 20:34:48 +0000 2009 

NO.3 user
id : 16676396
name : El Universal
screen_name : El_Universal_Mx
protected : False
verified : True
followers_count : 5512589
friends_count : 14014
listed_count : 25234
favourites_count : 29497
statuses_count : 864128
created_at : Fri Oct 10 00:09:06 +0000 2008 

PostgreSQL connection is closed


### 2. Find users with numbers of followers more than 100K

In [116]:
query_data("SELECT * FROM users WHERE followers_count>100000 ORDER BY followers_count DESC", "result")

There are 134 querying results 

NO.1 user
id : 1115874631
name : CGTN
screen_name : CGTNOfficial
protected : False
verified : True
followers_count : 14024195
friends_count : 56
listed_count : 8412
favourites_count : 68
statuses_count : 117674
created_at : Thu Jan 24 03:18:59 +0000 2013 

NO.2 user
id : 37034483
name : NDTV
screen_name : ndtv
protected : False
verified : True
followers_count : 12616711
friends_count : 15
listed_count : 12732
favourites_count : 0
statuses_count : 704225
created_at : Fri May 01 20:34:48 +0000 2009 

NO.3 user
id : 16676396
name : El Universal
screen_name : El_Universal_Mx
protected : False
verified : True
followers_count : 5512589
friends_count : 14014
listed_count : 25234
favourites_count : 29497
statuses_count : 864128
created_at : Fri Oct 10 00:09:06 +0000 2008 

NO.4 user
id : 24969337
name : Milenio
screen_name : Milenio
protected : False
verified : True
followers_count : 4561261
friends_count : 157
listed_count : 16320
favourites_count : 9
statuses

id : 18193312
name : Vanguardia
screen_name : vanguardiamx
protected : False
verified : True
followers_count : 199896
friends_count : 15975
listed_count : 1322
favourites_count : 2297
statuses_count : 334415
created_at : Wed Dec 17 17:03:43 +0000 2008 

NO.83 user
id : 25887412
name : 10 News First
screen_name : 10NewsFirst
protected : False
verified : True
followers_count : 199847
friends_count : 600
listed_count : 1056
favourites_count : 1557
statuses_count : 104235
created_at : Sun Mar 22 21:46:24 +0000 2009 

NO.84 user
id : 22677427
name : Michael Smerconish
screen_name : smerconish
protected : False
verified : True
followers_count : 197230
friends_count : 2101
listed_count : 2013
favourites_count : 1415
statuses_count : 23442
created_at : Tue Mar 03 20:52:50 +0000 2009 

NO.85 user
id : 16334139
name : WBZ | CBS Boston News
screen_name : wbz
protected : False
verified : True
followers_count : 194274
friends_count : 2791
listed_count : 3104
favourites_count : 517
statuses_count : 

### 3. Find the verified user with the smallest number of followers

In [117]:
query_data("SELECT * FROM users WHERE verified = true ORDER BY followers_count ASC LIMIT 1", "result")

There are 1 querying results 

NO.1 user
id : 545410392
name : Jen Kraschnewski, MD MPH
screen_name : jlkrasch
protected : False
verified : True
followers_count : 581
friends_count : 592
listed_count : 9
favourites_count : 2055
statuses_count : 775
created_at : Wed Apr 04 18:09:12 +0000 2012 

PostgreSQL connection is closed


### Delete the indexes

In [97]:
pgsql("DROP INDEX followers_idx","index")

Indexes created/dropped successfully in PostgreSQL

PostgreSQL connection is closed


In [98]:
pgsql("DROP INDEX followers_verified_idx","index")

Indexes created/dropped successfully in PostgreSQL

PostgreSQL connection is closed


## Delete the Table

In [100]:
pgsql("DROP TABLE IF EXISTS users;", "drop")

PostgreSQL table successfully dropped

PostgreSQL connection is closed


# MongoDB

# Please rememeber to put the path of file in the "with open section"

In [1]:
import pymongo
from pymongo import MongoClient
# you have to install mongodb using command line before using the code below

In [2]:
# client = MongoClient('localhost', 27017)
# db = client['twitter_db']
# collection = db['twitter_collection']
# import json
# with open("tweets", "r") as f1:
#     for line in f1:
#         try:
#             data = json.loads(line)
#             # if you want to see a specific field, you can print it. 
#             # if your file is big, there may be too many of these printed
#             # print(data['text'])
#             # insert data into MongoDB
#             collection.insert_one(data)
#         except:
#             continue

FileNotFoundError: [Errno 2] No such file or directory: 'tweets'

# Process Data in MongoDB

In [1]:
# import packages 
import pymongo
from pymongo import MongoClient

# Define client, db, collection
client = MongoClient('localhost', 27017)
db = client['twitter_db']
collection = db['twitter_collection']

# find one record in twitter_collection
x = collection.find_one()
# print(x)

In [2]:
# check how many tweets have been retweeted 
cursor = db.twitter_collection.find({"retweeted_status": {"$exists": True}})
num_of_tweets_retweeted = 0 
for doc in cursor:
    num_of_tweets_retweeted += 1
print(num_of_tweets_retweeted)

15633


## Create several indexes in this table for fast access

If not creating indexes, it may require traversal the whole table to find the information. With index, (MongoDB stores indexes using BTree data structure), the search time complexity can be reduced a lot. (Generally, can from O(N) to O(logN))

In [3]:
# create a single filed index for users' followers_count to speed up the access for each single tweet based 
# on the number of their followers
collection.create_index([("user.followers_count", pymongo.DESCENDING)])

# create an index on "created_at"
collection.create_index([("created_at", pymongo.DESCENDING)])

# create an index on "retweeted_status.retweet_count"
collection.create_index([("retweeted_status.retweet_count", pymongo.DESCENDING)])

# create an index on "retweeted_status.reply_count"
collection.create_index([("retweeted_status.reply_count", pymongo.DESCENDING)])

# look at executionTimeMillis for search time, the smaller executionTimeMillis is better 
# cursor.explain()

'retweeted_status.reply_count_-1'

## Create another collection named "first_100_tweets_table" 

In [4]:
# Create another collection named first_100_tweets_table
collection_2 = db['first_100_tweets_table']

### The following chunk of code can only be run once, and it insert the first 100 tweets into this collection one by one

In [5]:
#first_100_tweets = db.twitter_collection.find().sort("created_at", -1).limit(100)
#for doc in first_100_tweets:
#    collection_2.insert_one(doc)

In [6]:
# find one doc in the new collection "first_100_tweets_table"
y = collection_2.find_one()
#print(y)

## Make a deep copy of the first_100_tweets_table in this program, and it will be used as a in program database, which can be accessed faster than the database

In [7]:
# make a deep copy of the first_100_tweets_table in this program, and it can be accessed faster than the database
cache_in_program = []
cursor = db.first_100_tweets_table.find()
for doc in cursor:
    cache_in_program.append(doc)

## Make some queries using MongoDB

In [8]:
# find the number of tweets in this database 
cursor = db.twitter_collection.find()
num_of_tweets = 0 
for doc in cursor:
    num_of_tweets += 1
print(num_of_tweets)

19171


In [9]:
# find the content of the newest tweet
newest_tweet = db.first_100_tweets_table.find().limit(1) 
for doc in newest_tweet:
    content_of_newest_tweet = doc['text']
    print(content_of_newest_tweet)

RT @TarekFatah: Pakistanis in Karachi defying orders not to congregate in mosques by creating makeshift mosques on rooftops. Working hard t…


In [10]:
# find the time of the latest tweet created in this database
newest_tweet = db.first_100_tweets_table.find().limit(1) 
for doc in newest_tweet:
    created_time_of_newest_tweet = doc['created_at']
print(created_time_of_newest_tweet)

Wed Apr 15 00:56:34 +0000 2020


In [11]:
# find the user id of the user who has the largest number of followers in this database
user_with_largest_num_of_followers = db.twitter_collection.find({}, {"user.id", "user.name", "user.followers_count"}).sort("user.followers_count", -1).limit(1)
for doc in user_with_largest_num_of_followers:
    user_id_with_largest_num_of_followers = doc['user']['id']
    user_name_with_largest_num_of_followers = doc['user']['name']
print("user id: " + str(user_id_with_largest_num_of_followers) + "\nuser name: " + user_name_with_largest_num_of_followers)

user id: 1115874631
user name: CGTN


In [12]:
# find how many users in this database have more than 100k followers
cursor = db.twitter_collection.find({"user.followers_count": {"$gt": 100000}}, {"user.id", "user.followers_count"})
num_of_users_with_gt100k_followers = 0 
for doc in cursor:
    num_of_users_with_gt100k_followers += 1
print(num_of_users_with_gt100k_followers)

167


In [13]:
# find the tweets with most retweets
most_retweets = db.twitter_collection.find({}, {"retweeted_status.text", "retweeted_status.retweet_count"}).sort("retweeted_status.retweet_count", -1).limit(5)
for doc in most_retweets:
    tweets_with_most_retweets = doc['retweeted_status']['text']
    print(tweets_with_most_retweets)

All the other countries are making the US look like it’s being ran by a moron
All the other countries are making the US look like it’s being ran by a moron
All the other countries are making the US look like it’s being ran by a moron
All the other countries are making the US look like it’s being ran by a moron
All the other countries are making the US look like it’s being ran by a moron


In [14]:
# find tweets made by users who are verified
verified = db.twitter_collection.find({"user.verified": {"$exists": True}})
for doc in verified:
    verified_tweets = doc['text']
    verified_tweets_name = doc['user']['name']
    #print("Name:", verified_tweets_name, "\nTweet:", verified_tweets)

In [15]:
# get the average character length of all the tweets
all_tweets = db.twitter_collection.find()
sum = 0
for doc in all_tweets:
    tweet = doc['text']
    sum += len(tweet)
    
average_length = sum/num_of_tweets
print("The average length of a tweet in this database is",average_length, "characters.")

The average length of a tweet in this database is 123.95425382087528 characters.


In [16]:
# find tweets with the most replies (most controversial)
most_replies = db.twitter_collection.find({}, {"retweeted_status.text", "retweeted_status.reply_count"}).sort("retweeted_status.reply_count", -1).limit(5)
for doc in most_replies:
    length_of_tweets_with_most_replies = len(doc['retweeted_status']['text'])
    print(length_of_tweets_with_most_replies)
    tweets_with_most_replies = doc['retweeted_status']['text']
    print(tweets_with_most_replies)
    print()

140
Everyone, young and old, needs to act now to slow the spread of COVID-19.  The best thing Americans can to do fight… https://t.co/t9BooCw8QS

140
Everyone, young and old, needs to act now to slow the spread of COVID-19.  The best thing Americans can to do fight… https://t.co/t9BooCw8QS

140
Everyone, young and old, needs to act now to slow the spread of COVID-19.  The best thing Americans can to do fight… https://t.co/t9BooCw8QS

140
Preliminary investigations conducted by the Chinese authorities have found no clear evidence of human-to-human tran… https://t.co/1GHUbI2YXm

140
Preliminary investigations conducted by the Chinese authorities have found no clear evidence of human-to-human tran… https://t.co/1GHUbI2YXm



In [17]:
# # getting tweets that are blow a certain character length
# length = 40
# all_tweets = db.twitter_collection.find({}, {"retweeted_status", "retweeted_status.text"})
# for doc in all_tweets:
#     # if tweet is less than desired length, add it
#     if (len(doc['retweeted_status']['text']) < length):
#         short_tweets = doc['retweeted_status']['text']
#         #print(short_tweets)


In [18]:
# find tweets from people with the most followers
most_followers = db.twitter_collection.find({}, {"user.id", "user.followers_count", "text"}).sort("user.followers_count", -1).limit(2)
for doc in most_followers:
    user_id_with_most_followers = doc['user']['id']
    follower_count = doc['user']['followers_count']
    tweets_by_most_followers = doc['text']
    tweets_from_people_with_most_followers = "User ID: " + str(user_id_with_most_followers) + "\nNumber of followers: " + str(follower_count) + "\nTweet: " + tweets_by_most_followers
    print(tweets_from_people_with_most_followers)

User ID: 1115874631
Number of followers: 14024195
Tweet: Numbers from the Chinese mainland on Tuesday: one new #COVID19 death in #Hubei; 46 new cases (36 originating abroad… https://t.co/Fw6RKZoDQf
User ID: 37034483
Number of followers: 12616711
Tweet: Here's how drones helping India in fight against #coronavirus https://t.co/ATdiBectqs https://t.co/Cx6dH0Hc2O


## Prepare answers to some common questions in the program which will be put into cache later 

In [19]:
common_questions = {
    "Number of tweets in database?": num_of_tweets,
    "What is the content of the newest tweet?": content_of_newest_tweet,
    "What time is the latest tweet created in this database?": created_time_of_newest_tweet,
    "What is the user id of the user who has the largest number of followers in this database?": "user id: " + str(user_id_with_largest_num_of_followers) + "\nuser name: " + user_name_with_largest_num_of_followers,
    "What is the content of the tweet with most retweets? ": tweets_with_most_retweets,
    "How many users in this database have more than 100k followers?": num_of_users_with_gt100k_followers,
    "What is the average length of a tweet in this database?": average_length,
    "What is the tweet with most replies, and how many replies it gets?": tweets_with_most_replies + "\n" + str(length_of_tweets_with_most_replies) + " replies",
    "What is the tweet from people with most followers? ": tweets_from_people_with_most_followers
    
}

In [20]:
for i, val in common_questions.items():
    print(i, val)

Number of tweets in database? 19171
What is the content of the newest tweet? RT @TarekFatah: Pakistanis in Karachi defying orders not to congregate in mosques by creating makeshift mosques on rooftops. Working hard t…
What time is the latest tweet created in this database? Wed Apr 15 00:56:34 +0000 2020
What is the user id of the user who has the largest number of followers in this database? user id: 1115874631
user name: CGTN
What is the content of the tweet with most retweets?  All the other countries are making the US look like it’s being ran by a moron
How many users in this database have more than 100k followers? 167
What is the average length of a tweet in this database? 123.95425382087528
What is the tweet with most replies, and how many replies it gets? Preliminary investigations conducted by the Chinese authorities have found no clear evidence of human-to-human tran… https://t.co/1GHUbI2YXm
140 replies
What is the tweet from people with most followers?  User ID: 37034483
Numbe

# LRU Cache for questions asked
### LRU Cache is a cache replacement algorithm that removes the least recently used data in order to make room for new data.

In [21]:
# LRU Cache requires the linkedlist structure, but Python does not have it, so we have to create a LinkedNode class
# key is the question, value is the answer to that question, next is the next question of this question in the cache
class LinkedNode:
    
    def __init__(self, key=None, value=None, next=None):
        self.key = key
        self.value = value
        self.next = next

In [22]:
# LRU Cache is a cache replacement algorithm that removes the least recently used data in order to make room for 
# new data.
class LRUCache:
    
        # This is the initial function to define 
        # Dictionary(key_to_prev): key is the question, value is the linkednode of the previous question
        # LinkedNode(dummy): the linkednode without value to point out the memory address of the linkedlist
        # tail: the tail of the linkedlist, and its initial value is dummy
        # Integer(capacity): define of the size of the cache, which is the same as the length of the linkedlist
        def __init__(self, capacity):
            self.key_to_prev = {}
            self.dummy = LinkedNode()
            self.tail = self.dummy
            self.capacity = capacity
        
        # The function push_back is to put the question node at the tail of the linkedlist
        def push_back(self, node):
            self.key_to_prev[node.key] = self.tail
            self.tail.next = node
            self.tail = node
        
        # The function pop_front is to delete the head node from the linkedlist and the next node becomes the new head
        def pop_front(self):
            head = self.dummy.next
            del self.key_to_prev[head.key]
            self.dummy.next = head.next
            self.key_to_prev[head.next.key] = self.dummy
            
        # The function kick is to move the prev node's next to the tail of the linkedlist
        def kick(self, prev): 
            node = prev.next
            if node == self.tail:
                return
        
            # remove the current node from linked list
            prev.next = node.next
            # update the previous node in hash map
            self.key_to_prev[node.next.key] = prev
            node.next = None

            self.push_back(node)
        
        # The function get is to get the value(answer) of the key(question)
        # If the question is not in the linkedlist, it will return -1, and we need go to the database to find answers
        # Else: it will return the value(answer) of the key(question)
        def get(self, key):
            if key not in self.key_to_prev:
                return -1
        
            prev = self.key_to_prev[key]
            current = prev.next
        
            self.kick(prev)
            return current.value
        
        # The function put is to put a question into the linkedlist
        def set(self, key, value):
            # If the question is in the cache, it will move the question from the original node position to the 
            # tail of the original linkedlist
            if key in self.key_to_prev:
                self.kick(self.key_to_prev[key])
                self.key_to_prev[key].next.value = value
                return
            
            # If the question is not in cache, it will be inseretd to the linkedlist
            self.push_back(LinkedNode(key, value))
            # In addition, if the cache reached its capacity, it will invaliaded the lease recently used question 
            # before inserting the new question 
            if len(self.key_to_prev) > self.capacity:
                self.pop_front()     

## Apply LRU Cache algorithm to cache of questions asked in this project

In [23]:
if __name__ == "__main__":
    # set the size of the cache be 20 questions
    LRU_Cache = LRUCache(20)

    # put several common questions into cache firstly
    for question in common_questions:
        LRU_Cache.set(question, common_questions[question])

In [24]:
LRU_Cache.get("Number of tweets in database?")

19171

In [25]:
LRU_Cache.get("What is the content of the newest tweet?")

'RT @TarekFatah: Pakistanis in Karachi defying orders not to congregate in mosques by creating makeshift mosques on rooftops. Working hard t…'

In [26]:
LRU_Cache.get("What is LRU Cache?")

-1

# Search Application

## Initial attempt: Design a basic serach application UI with the questions and answers based on common_questions

In [27]:
# import some packages for user interface design 
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider

# define a function f to return the answer to the input question 
def f(Question):
    print(common_questions[Question])

In [30]:
# interface for user to do question search 
def search_application():
    interact(f, Question = [x for x in common_questions])
search_application()
# import timeit
# print(timeit.timeit(search_application, number=1))

interactive(children=(Dropdown(description='Question', options=('Number of tweets in database?', 'What is the …

In [31]:
# w = widgets.Dropdown(
#     options = common_questions,
#     description = 'Please pick a question:', 
#     style = style, 
#     layout = {'width': 'max-content'})
# display(w)

## Check running time of the common questions for search application

In [30]:
def common_question_in_cache():
    print(common_questions["What is the content of the newest tweet?"])

import timeit
print(timeit.timeit(common_question_in_cache, number=1))

RT @TarekFatah: Pakistanis in Karachi defying orders not to congregate in mosques by creating makeshift mosques on rooftops. Working hard t…
5.926200000061499e-05


In [32]:
def common_question_outof_cache():
    newest_tweet = db.first_100_tweets_table.find().limit(1) 
    for doc in newest_tweet:
        content_of_newest_tweet = doc['text']
        print(content_of_newest_tweet)
import timeit
print(timeit.timeit(common_question_outof_cache, number=1))

RT @TarekFatah: Pakistanis in Karachi defying orders not to congregate in mosques by creating makeshift mosques on rooftops. Working hard t…
0.0024367979999624367


In [31]:
def common_question_in_cache():
    print(common_questions["Number of tweets in database?"])

import timeit
print(timeit.timeit(common_question_in_cache, number=1))

19171
4.860399999984111e-05


In [33]:
def common_question_outof_cache():
    cursor = db.twitter_collection.find()
    num_of_tweets = 0 
    for doc in cursor:
        num_of_tweets += 1
    print(num_of_tweets)
import timeit
print(timeit.timeit(common_question_outof_cache, number=1))

19171
3.3622609230001217


## Improvement on Search Application based on LRU Cache and More Questions 

In [34]:
import pandas as pd
import numpy as np

In [35]:
# NewTweet is to find the created_time, user_id, user_name and content of newest tweets
class NewTweet:
    
    def __init__(self, number):
        self.number = number
    
    def find_tweet_content(self):
        newest_tweet = db.first_100_tweets_table.find().limit(self.number)
        index = 1
        result = []
        for doc in newest_tweet:
            temp = []
            content = doc['text']
            created_time = doc['created_at']
            user_id = doc['user']['id']
            user_name = doc['user']['name']
            temp.append("No. " + str(index))
            temp.append(created_time + " User ID: " + str(user_id) + " User Name: " + user_name)
            temp.append(content_of_newest_tweet)
            result.append(temp)
            index += 1
        return result

# FamousUsers is to find the user_id, user_name and number_of_followers of users with most followers
class FamousUsers:
    def __init__(self, number):
        self.number = number
    
    def find_famous_users(self):
        # find the user id of the user who has the largest number of followers in this database
        user_with_largest_num_of_followers = db.twitter_collection.find({}, {"user.id", "user.name", "user.followers_count"}).sort("user.followers_count", -1).limit(self.number)
        index = 1
        result = []
        for doc in user_with_largest_num_of_followers:
            temp = []
            user_id = doc['user']['id']
            user_name = doc['user']['name']
            number_of_followers = doc['user']['followers_count']
            temp.append("No. " + str(index))
            temp.append("User ID: " + str(user_id) + "\nUser Name: " + user_name + "\nFollowers: " + str(number_of_followers))
            result.append(temp)
            index += 1
        return result

            
from collections import Counter
# PopularWords is to find words which have been used most in this dataset
class PopularWords:
    def __init__(self, number):
        self.number = number
    
    def find_popular_words(self):
        sentences = db.twitter_collection.find({}, {"text"})
        word_dictionary = {}
        for doc in sentences:
            sentence = doc['text']
            for word in sentence.split():
                if word in word_dictionary:
                    word_dictionary[word] += 1
                else:
                    word_dictionary[word] = 1
        k = Counter(word_dictionary)
        high = k.most_common(self.number)
        index = 1
        result = []
        for word in high:
            temp = []
            temp.append("No. " + str(index))
            temp.append(word[0] + ": " + str(word[1]))
            result.append(temp)
            index += 1
        return result

# TweetsFromFamousUsers is to find tweets from people with the most followers
class TweetsFromFamousUsers:
    def __init__(self, number):
        self.number = number
        
    def find_tweets_from_famous_users(self):
        most_followers = db.twitter_collection.find({}, {"user.id", "user.followers_count", "text"}).sort("user.followers_count", -1).limit(self.number)
        index = 1
        result = []
        for doc in most_followers:
            user_id = doc['user']['id']
            follower_count = doc['user']['followers_count']
            tweet = doc['text']
            tweets_from_famous_users = "User ID: " + str(user_id) + "\nNumber of Followers: " + str(follower_count) + "\nTweet: " + tweet
            temp = []
            temp.append("No. " + str(index))
            temp.append(tweets_from_famous_users)
            result.append(temp)
            index += 1
        return result

# ValidQuestion is to find the question to its related query 
class ValidQuestion:
    
    def __init__(self, question, number):
        self.question = question
        self.number = number
    
    def get_question(self):
        if self.question == "Find Newest Tweets":
            new_tweet = NewTweet(self.number)
            return new_tweet.find_tweet_content()
        
        elif self.question == "Find Famous Users":
            famous_users = FamousUsers(self.number)
            return famous_users.find_famous_users()

        elif self.question == "Find popular words":
            popular_words = PopularWords(self.number)
            return popular_words.find_popular_words()
        
        elif self.question == "Find tweets from famous users":
            tweets_from_famous_users = TweetsFromFamousUsers(self.number)
            return tweets_from_famous_users.find_tweets_from_famous_users()
        
        else:
            print("We don't have this question in search application!")
            return

In [36]:
import ipywidgets as widgets
dropdown_count = widgets.Dropdown(options = [x for x in range(100)])

In [37]:
def dropdown_count(change):
    valid_question 
    display(df_london[df_london.year == change.new])

In [38]:
valid_question = ValidQuestion("Find Newest Tweets", 5)
valid_question.get_question()

[['No. 1',
  'Wed Apr 15 00:56:34 +0000 2020 User ID: 22091137 User Name: Basu Ghosh Das',
  'RT @TarekFatah: Pakistanis in Karachi defying orders not to congregate in mosques by creating makeshift mosques on rooftops. Working hard t…'],
 ['No. 2',
  'Wed Apr 15 00:56:34 +0000 2020 User ID: 531629036 User Name: Creeds Cannon',
  'RT @TarekFatah: Pakistanis in Karachi defying orders not to congregate in mosques by creating makeshift mosques on rooftops. Working hard t…'],
 ['No. 3',
  'Wed Apr 15 00:56:34 +0000 2020 User ID: 1042203452212948992 User Name: christy��️\u200d��',
  'RT @TarekFatah: Pakistanis in Karachi defying orders not to congregate in mosques by creating makeshift mosques on rooftops. Working hard t…'],
 ['No. 4',
  'Wed Apr 15 00:56:34 +0000 2020 User ID: 3551287573 User Name: david lee',
  'RT @TarekFatah: Pakistanis in Karachi defying orders not to congregate in mosques by creating makeshift mosques on rooftops. Working hard t…'],
 ['No. 5',
  'Wed Apr 15 00:56:33 +0

In [56]:
# from IPython.display import display
class SearchApplicationOne:
    def __init__(self, capacity):
        self.LRU_Cache = LRUCache(capacity)
    
    def user_interface(self):
        @interact(options = widgets.Dropdown(options=['Find Newest Tweets', 'Find Famous Users', 'Find popular words', 
                                              'Find tweets from famous users'],
            value='Find Newest Tweets',
            description='Question:',
            disabled=False), 
            Top = widgets.IntSlider(min = 1, max = 100, step = 1, description = "Top: ", value = 1))

        def f(options, Top):
            if self.LRU_Cache.get((options, Top)) != -1:
                answers = self.LRU_Cache.get((options, Top))
                for answer in answers:
                    for val in answer:
                        print(val)
                    print()
                    
            else:
                valid_question = ValidQuestion(options, Top)
                answers = valid_question.get_question()
                self.LRU_Cache.set((options, Top), answers)
                for answer in answers:
                    for val in answer:
                        print(val)
                    print()
            print(self.LRU_Cache.key_to_prev)

In [57]:
serach_appliaction = SearchApplicationOne(10)
serach_appliaction.user_interface()

interactive(children=(Dropdown(description='Question:', options=('Find Newest Tweets', 'Find Famous Users', 'F…